In [2]:
import json
import requests
from bs4 import BeautifulSoup

# Load Login Credentials

In [11]:
with open('credentials.json', 'r') as f:
    credentials = json.load(f)
    username = credentials['USERNAME']
    password = credentials['PASSWORD']
    
URL = 'https://www.mytwinoaks.com/MemberPortal/Login.aspx?clubid=1699'
headers={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.125 Safari/537.36"}

# Start Session and Login

In [13]:
print("Getting:", URL)
s=requests.Session()
s.headers.update(headers)

r=s.get(URL)
soup=BeautifulSoup(r.content)

VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']

data={"__VIEWSTATE":VIEWSTATE,
"__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
"__EVENTVALIDATION":EVENTVALIDATION,
"ctl00$PageContent$LoginName":username,
"ctl00$PageContent$PassWord":password,
"ctl00$PageContent$loginButton":"Log In"}

r = s.post(URL, data)
URL = r.url
soup = BeautifulSoup(r.content)
print("Returned:",URL)

Getting: https://www.mytwinoaks.com/MemberPortal/Login.aspx?clubid=1699
Returned: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699


# Get a history of visits

## Navigate to Usage Page

In [14]:
VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']

data = {"__VIEWSTATE":VIEWSTATE,
        "__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
        "__VIEWSTATEENCRYPTED":"",
        "__EVENTVALIDATION":EVENTVALIDATION,
        "ctl00$IsLoggedIn":1,
        "ctl00$PageContent$amtToPayPopupBox":0,
        "__EVENTTARGET":"ctl00$PageSubNavBar$UsageButton",
        "__EVENTARGUMENT":""
       }

print("Getting:", URL)
r = s.post(URL, data)
URL = r.url
soup = BeautifulSoup(r.content)
print("Returned:",URL)

Getting: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699
Returned: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699


## Open Printable View of Usage

In [15]:
#First, trigger the ability to GET

VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']

data = {"__VIEWSTATE":VIEWSTATE,
        "__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
        "__VIEWSTATEENCRYPTED":"",
        "__EVENTVALIDATION":EVENTVALIDATION,
        "ctl00$IsLoggedIn":1,
        "ctl00$PageContent$fromDateBox":"",
        "ctl00$PageContent$toDateBox:":"",
        "ctl00$PageContent$amtToPayPopupBox":0,
        "__EVENTTARGET":"ctl00$PageContent$usePrintableLink",
        "__EVENTARGUMENT":""
       }

print("Getting:", URL)
r = s.post(URL, data)
URL = r.url
soup = BeautifulSoup(r.content)
print("Returned:",URL)

Getting: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699
Returned: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699


## Record Data

In [16]:
checkins = s.get("https://www.mytwinoaks.com/MemberPortal/ClubServices/MemberUsageReport.aspx")

with open('Checkins.html', 'w') as w:
    w.write(checkins.text)

# Get a history of payments

## Naviate to the payment page

In [17]:
VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']

data = {"__VIEWSTATE":VIEWSTATE,
        "__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
        "__VIEWSTATEENCRYPTED":"",
        "__EVENTVALIDATION":EVENTVALIDATION,
        "ctl00$IsLoggedIn":1,
        "ctl00$PageContent$amtToPayPopupBox":0,
        "__EVENTTARGET":"ctl00$PageSubNavBar$ARHistoryButton",
        "__EVENTARGUMENT":""
       }

print("Getting:", URL)
r = s.post(URL, data)
URL = r.url
soup = BeautifulSoup(r.content)
print("Returned:",URL)

Getting: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699
Returned: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699


## Open printable view of Payments

In [18]:
VIEWSTATE=soup.find(id="__VIEWSTATE")['value']
VIEWSTATEGENERATOR=soup.find(id="__VIEWSTATEGENERATOR")['value']
EVENTVALIDATION=soup.find(id="__EVENTVALIDATION")['value']

data = {"__VIEWSTATE":VIEWSTATE,
        "__VIEWSTATEGENERATOR":VIEWSTATEGENERATOR,
        "__VIEWSTATEENCRYPTED":"",
        "__EVENTVALIDATION":EVENTVALIDATION,
        "ctl00$IsLoggedIn":1,
        "ctl00$PageContent$fromDateBoxAR":"",
        "ctl00$PageContent$toDateBoxAR:":"",
        "ctl00$PageContent$amtToPayPopupBox":0,
        "__EVENTTARGET":"ctl00$PageContent$ARPrintableLink",
        "__EVENTARGUMENT":""
       }

print("Getting:", URL)
r = s.post(URL, data)
URL = r.url
soup = BeautifulSoup(r.content)
print("Returned:",URL)

Getting: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699
Returned: https://www.mytwinoaks.com/MemberPortal/ClubServices/memDashboard.aspx?clubid=1699


## Record Data

In [19]:
payments = s.get("https://www.mytwinoaks.com/MemberPortal/ClubServices/ARHistoryReport.aspx")

with open('Payments.html', 'w') as w:
    w.write(payments.text)

# Scrape Downloaded Data

In [3]:
import datetime
import pandas as pd
import numpy as np
import calendar

#### *Visits* is a series of datetime objects in sorted order (early to late) that corresponds to check-ins.

In [22]:
#Checkins
with open('Checkins.html', 'r') as f:
    soup = BeautifulSoup(f)

visits = []
for record in [rec for rec in soup.find_all('tr') if len(rec.find_all('span'))==5]:
    club, _, time, _, alerts = [rec.text for rec in record.find_all('span')]
    visits.append(datetime.datetime.strptime(time, '%m/%d/%Y %I:%M:%S %p'))
        
visits = pd.Series(visits)
visits.sort()
visits = visits.reset_index(drop=True)

visits.head()

0   2009-10-10 14:30:45
1   2009-10-12 18:07:58
2   2009-10-13 18:26:28
3   2009-10-17 11:35:13
4   2009-10-21 17:59:35
dtype: datetime64[ns]

#### *Payments* is a dataframe of payment records

In [31]:
#Payments
with open('Payments.html', 'r') as f:
    soup = BeautifulSoup(f)

payments = {'date':[],'debit':[]}
for record in [rec for rec in soup.find_all('tr') if len(rec.find_all('span'))==11]:
    info = [rec.text for rec in record.find_all('span')]
    date = info[0]
    debit = info[6]
    payments['date'].append(datetime.datetime.strptime(date, '%m/%d/%Y'))
    payments['debit'].append(float(debit))
    
payments = pd.DataFrame(payments)
payments = payments.sort('date').reset_index(drop=True) #Sort from earliest and re-index

print(payments.head())
print(payments.dtypes)

        date  debit
0 2009-10-10     25
1 2009-10-10     21
2 2009-11-01     29
3 2009-12-01     29
4 2010-01-01     29
date     datetime64[ns]
debit           float64
dtype: object


# Fun with data!

In [32]:
total_payments = payments['debit'].sum()
total_visits = len(visits)
print('Total Payments:', '${0:,.2f}'.format(total_payments))
print('Total Visits:', '{0:,}'.format(total_visits))
print('Average cost per visit:', "${0:,.2f}".format(total_payments/total_visits))

Total Payments: $1,960.00
Total Visits: 961
Average cost per visit: $2.04


In [36]:
#Most Visits per Year, Month-Year, Weekday

In [37]:
#Longest Break
timediff = visits.shift(-1)-visits #time until next visit
maxbreak = timediff.max()
start = visits.loc[timediff.idxmax()].strftime("%B %d, %Y @ %I:%M %p")
end = visits.loc[timediff.idxmax()+1].strftime("%B %d, %Y @ %I:%M %p")
print("Longest break was from {} to {}".format(start, end))
print("This was {} Days, {} Hours, {} Min, and {} Sec".format(maxbreak.days, maxbreak.seconds//3600,
                                                              maxbreak.seconds//60%60, maxbreak.seconds%60))

Longest break was from August 29, 2012 @ 06:57 PM to September 16, 2012 @ 10:58 AM
This was 17 Days, 16 Hours, 0 Min, and 54 Sec


In [38]:
#Longest Streak
dates = visits.apply(lambda dt: dt.date())
daydiff = dates.shift(-1)-dates #days until next visit

#Select only rows with consecutive dates
consecutive =daydiff==pd.Timedelta('1 days')
print(consecutive.head())
#Find longest stretch of consecutive index numbers

0    False
1     True
2    False
3    False
4    False
dtype: bool


# Fun with plots!

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook")
%matplotlib inline

## Overview

## Interactions